In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
from nltk import flatten
import numpy as np
import pandas as pd
import numpy as np
import torch
import torchtext
from torchtext import data
import torch.optim as optim
import tensorflow as tf
import argparse
import os
import torch.nn.functional as F
import torch.nn as nn
import logging
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# import gensim.downloader as api
torch.cuda.empty_cache()
np.random.seed(43)
torch.manual_seed(43)

In [ ]:
data = pd.read_csv('cleaned_tweets.csv')
emoji_data = pd.read_csv('emoji.csv')

In [ ]:
tweets_posts=[]
target_number=[]
target_emoji=[]
num_emoji=2000
#take 2000 examples for now
for e in emoji_data['symbol']:
  tweets_posts.append(data[data.emoji==e]['tweets'][:num_emoji].tolist()) #find the first 2000 (subject to change) tweets of emoji e
  target_number.append([emoji_data[emoji_data['symbol']==e].index.values]*num_emoji) #append the index of emoji e 250 times (subject to change) to a list
  target_emoji.append([e]*num_emoji)

#they are nested list so need to flatten them
target_number=flatten(target_number)
target_number=[i[0] for i in target_number]
target_emoji=flatten(target_emoji)
tweets_posts=flatten(tweets_posts)

#create a dictionary
data2 = {'tweets':tweets_posts,
        'emojis_num':target_number,
         'emoji':target_emoji}

In [ ]:
#convert dictionary to a dataframe
tweets_dataset = pd.DataFrame(data2)
tweets_dataset

,tweets,emojis_num,emoji
0,love to see it the numbers they both pull is m...,0,👏
1,a story book scene congrats to both of you,0,👏
2,four arms more like for us,0,👏
3,here here an amazing team of reps delivering a...,0,👏
4,they should do that to all cex,0,👏
...,...,...,...
119995,no no tonic getaway cory ranco full results an...,59,▶️
119996,this is not last dance for messi ronaldo on th...,59,▶️
119997,with im so into you swv,59,▶️
119998,at saintcloud stormy night rue jonas mi cinfil...,59,▶️


In [ ]:
train,test0=train_test_split(tweets_dataset,test_size=0.36,random_state=1,stratify=tweets_dataset['emojis_num'])
valid,test=train_test_split(test0,test_size=20/36,random_state=1,stratify=test0['emojis_num'])
train=train.reset_index(drop=True)
valid=valid.reset_index(drop=True)
test=test.reset_index(drop=True)
# a=train[train['emojis_num']==2]
# a.info()

In [ ]:
# TextDataset is Described in Section 3.3 of Assignment 2

class TextDataset(torch.utils.data.Dataset):
    def __init__(self,vocab,df):
        # data_path = "data"
        # df = pd.read_csv(os.path.join(data_path, f"{split}.tsv"), sep="\t")
       
        # X: torch.tensor (maxlen, batch_size), padded indices
        # Y: torch.tensor of len N
        X, Y = [], []
        V = len(vocab.vectors)
        for i, row in df.iterrows():
            
            L = row["tweets"].split()
            
            # L = re.sub('[0-9]+', '', row['tweets']) 
            # L=re.split('[ ,.!?:$&*/\|<>^+=-_-•%;。･，<>]',L)  
            # L = L.split()
            # no_extraspace = [" ".join(i.split()) for i in no_newlinechar]
            # L=[j for j in L if j]
            # print("original",row["tweets"],"L",L)
            X.append(torch.tensor([vocab.stoi.get(w, V-1) for w in L]))  # Use the last word in the vocab as the "out-of-vocabulary" token
            temp=row["emojis_num"]
            temp=torch.tensor(temp)
            temp=F.one_hot(temp,num_classes=60)
            # print("Y",temp)
            Y.append(temp)
            # Y.append(row["emojis_num"])
        self.X = X 
        self.Y = Y
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx] 


In [ ]:
glove=torchtext.vocab.GloVe(name='840B',dim=300)
# glove=torchtext.vocab.GloVe(name='twitter.27B',dim=200)

In [ ]:
# my_collate_function prepares batches
# it also pads each batch with zeroes.
def my_collate_function(batch, device):
    # Handle the padding here
    # batch is approximately: [dataset[i] for i in range(0, batch_size)]
    # Since the dataset[i]'s contents is defined in the __getitem__() above, this collate function 
    # should be set correspondingly.
    # Also: collate_function just takes one argument. To pass in additional arguments (e.g., device), 
    # we need to wrap up an anonymous function (using lambda below)
    batch_x, batch_y = [], []
    max_len = 0
    for x,y in batch:
        # print('X',x,"Y",y)
        batch_y.append(y)
        max_len = max(max_len, len(x))#longest sentence in a batch
    for x,y in batch:
      #if sentence not longest, set rest token as 0
        x_p = torch.concat(
            [x, torch.zeros(max_len - len(x))]
        )
        batch_x.append(x_p)
    # print("batchy",batch_y)
    X=torch.stack(batch_x).t().int().to(device)
    # print("a",a)
    Y=torch.stack(batch_y).t().int().to(device)
    # print("b",b)
    return X,Y


In [ ]:
class Baseline(nn.Module):#Baseline model

    def __init__(self, embedding_dim, vocab):
        super(Baseline, self).__init__()
        #100 vectors for each word 
        #embedding.shape=[embedding dim,glove.len]
        self.embedding = nn.Embedding.from_pretrained(vocab.vectors)
        self.fc = nn.Linear(embedding_dim, 60)#60 emojis used

    def forward(self, x, len=None):
        #x has shape [sentence length, batch size]
        embedded = self.embedding(x)#embedded.shape=[sentence len, batch size,embedding size]
        # print("embedded",embedded.shape)
        average = embedded.mean(0) # average.shape=[batch size, embedding size]
        # print("average",average.shape)
        prediction = self.fc(average) #prediciton shape (bsz,60)
        # print("prediction",prediction.shape)
        # prediction=torch.nn.Softmax()(prediction)
        # prediction=F.log_softmax(prediction,dim=1)#prediction.shape(bsz,60)
        # print("lof sofamax",prediction.shape)
        return prediction

In [ ]:
def evaluate(model,dataloader,criterion):
    total_corr=0
    iterator=iter(dataloader)
    temp_loss=0
    for n, (input,labels) in enumerate(iterator):
      labels=labels.reshape(-1,60)
      # print("labels",labels.shape)
      prediction=model(input,input.size(dim=0))
      # prediction=F.log_softmax(prediction,dim=1)
      # print("predict",prediction.shape)
      loss=criterion(input=prediction.float(),target=labels.float())
      #Count correct examples
      k=8
      prediction=F.softmax(prediction,dim=1)
      _,pre=torch.topk(prediction,k,dim=1)
      true=torch.argmax(labels,dim=1)
      # corr=torch.count_nonzero(torch.eq(true,pre))
      for i in range(len(true)):
        # print("true[i",true[i])
        # print("predict i",pre[i])
        if true[i] in pre[i]:
          # print("+++++++++++++++++")
          total_corr=total_corr+1
          # print("total orr",total_corr)
 
      # total_corr=total_corr+int(corr.sum())
      temp_loss=loss+temp_loss 
    accuracy=float(total_corr)/len(dataloader.dataset)
    loss=(temp_loss/len(iterator))
    #print("validation accuracy",accuracy)
    return accuracy,loss

In [ ]:
#Baseline code start
def main(args):
    #   fix seed
    torch.manual_seed(2)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print ("Using device:", device)

    ### 3.3 Processing of the data ###
    # 3.3.1
    # The first time you run this will download a 862MB size file to .vector_cache/glove.6B.zip
    # glove = torchtext.vocab.GloVe(name="6B",dim=100) # embedding size = 100
    glove=torchtext.vocab.GloVe(name='840B',dim=300)
    # glove=torchtext.vocab.GloVe(name='twitter.27B',dim=200)
                                   
    # 3.3.2

    train_dataset=TextDataset(glove,train)
    val_dataset = TextDataset(glove,valid)
    test_dataset = TextDataset(glove, test)
        
    # 3.3.3
    #each dataloader is a batch
    train_dataloader = torch.utils.data.DataLoader(
        dataset=train_dataset, 
        batch_size=args.batch_size, 
        shuffle=False, 
        collate_fn=lambda batch: my_collate_function(batch, device))

    validation_dataloader = torch.utils.data.DataLoader(
        dataset=val_dataset, 
        batch_size=args.batch_size, 
        shuffle=False, 
        collate_fn=lambda batch: my_collate_function(batch, device))
   # print(args)
    test_dataloader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=lambda batch: my_collate_function(batch, device))

    #Constants
    embedding_size=300
    epochs=40
    lr=0.0001
    criterion=torch.nn.CrossEntropyLoss()  

    #training
    model=Baseline(embedding_size,glove).to(device)
    optimizer=torch.optim.Adam(model.parameters(), lr)
    epoch_list=[]
    train_loss=[]
    accuracy_train=[]
    accuracy_valid=[]
    accuracy_test=[]
    valid_loss=[]
    test_loss=[]

    for epoch in range(epochs):
        print("epoch",epoch)
        train_iter=iter(train_dataloader)
        valid_iter=iter(validation_dataloader)
        test_iter=iter(test_dataloader)
     
       
        # a,b,c=model.parameters()
        # print(b)
        # print("-----------------------------------------------------------------------------------------------")
        epoch_list.append(epoch+1)
        

        ##TRAINING SET
        for n, (input,labels) in enumerate(train_iter):
            # print("current epoch",epoch)
            input=input.to(device)
            labels=labels.to(device)
            labels=labels.reshape(-1,60)
            # print("n",n)
            # print("labels",labels.shape)
            prediction=model(input,input.size(dim=0))
            # print("prediciton",prediction.shape)
            # print("prediction",prediction)
            # print("true labels",labels)
            optimizer.zero_grad()
            torch.cuda.empty_cache()
            loss=criterion(input=prediction.float(),target=labels.float())
            # print("------------------------------------------")
            # print("loss",loss)
            #backward pass and update gradient
            loss.backward()
            optimizer.step()
           
            # #Count correct examples
            # pre=torch.argmax(prediction,dim=1)
            # true=torch.argmax(labels,dim=1)
            # corr=torch.count_nonzero(torch.eq(true,pre))
            # print("pre",pre)
            # print("true",true)
            # print("correct number",corr)

            # corr = ((prediction > 0.5).squeeze().long() == labels.long())
            # total_corr_train=total_corr_train+int(corr.sum())
            # temp_loss=loss+temp_loss
                 
        # # accuracy=float(total_corr_train)/len(train_dataset)
        accuracy,t_loss=evaluate(model,train_dataloader,criterion)
        print("loss",loss)
        print("accuracy",accuracy)
        train_loss.append(t_loss)
        accuracy_train.append(accuracy)
        if (epoch + 1) % 10 == 0:
          print("-----------------------------------------------------------------------------------------------")
          print('Epoch:', '%03d' % (epoch + 1), 'accuracy =', accuracy_train)

     

        # #validation #for 4.4
        # # print("before go to valid",temp_loss)
        # for n, (input,labels) in enumerate(valid_iter):
        #     # print("current epoch",epoch)
        #     prediction=model(input,input.size(dim=0)) 
        #     loss=criterion(input=prediction.float().squeeze(),target=labels.float().squeeze())
            
        #     
        #     # # #Count correct examples
        #     # corr = ((prediction > 0.5).squeeze().long() == labels.long())
        #     # total_corr_valid=total_corr_valid+int(corr.sum())
           
        #     temp_loss=loss+temp_loss  
        #     # print("temp loss",temp_loss)  
        # #print("validation length",len(val_dataset)) 
        # # accuracy=float(total_corr_valid)/len(val_dataset)
        # valid_loss.append(temp_loss/len(valid_iter))
        
        ###4.5 Validation loss, accuracy
        accuracy,v_loss=evaluate(model,validation_dataloader,criterion)
        valid_loss.append(v_loss)
        accuracy_valid.append(accuracy)
        #Test Loss, accuracy
       
        t_acc,t_loss=evaluate(model,test_dataloader,criterion)
        test_loss.append(t_loss)
        accuracy_test.append(t_acc)

    ####4.6
    # a,b,c=model.parameters()
    torch.save(model.cpu().state_dict(),'/content/drive/MyDrive/Colab Notebooks/model_baseline.pt')
    model.cuda()
        
    print("epoch list",epoch_list)

  #  ##Train dataset
    train_loss=torch.stack(train_loss)
    train_loss=train_loss.detach().cpu().clone().numpy()   
    print("train loss",train_loss)
    print("train accuracy list",accuracy_train)
    ##Validation dataset
    valid_loss=torch.stack(valid_loss)
    valid_loss=valid_loss.detach().cpu().clone().numpy() 
    print("valid accuracy list",accuracy_valid)
    print("valid loss",valid_loss)
    ##Test dataset
    test_loss=torch.stack(test_loss)
    test_loss=test_loss.detach().cpu().clone().numpy() 
    print("test accuracy list",accuracy_test)
    print("test loss",test_loss)


    plt.plot(epoch_list,train_loss,color='magenta',label="training error")
    plt.plot(epoch_list,valid_loss,color='blue',label="validation error")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Loss vs. Epoch")
    plt.legend(loc="upper right")
    plt.show()

    plt.plot(epoch_list,accuracy_train,color='magenta',label="training accuracy")
    plt.plot(epoch_list,accuracy_valid,color='blue',label="validation accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs. Epoch")
    plt.legend(loc="upper right")
    plt.show()
    
    ##Plot test accuracy
    plt.plot(epoch_list,accuracy_test,color='magenta',label="test accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs. Epoch")
    plt.legend(loc="upper right")
    plt.show()

    ##plot test loss
    plt.plot(epoch_list,test_loss,color='magenta',label="test loss")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.title("Error vs. Epoch")
    plt.legend(loc="upper right")
    plt.show()
    return model
  #   for param_tensor in model.state_dict():
  #     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('-f')
  parser.add_argument('--batch-size', type=int, default=5)
  args=parser.parse_args()
  main(args)

Using device: cuda
epoch 0
loss tensor(4.0985, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.115078125
epoch 1
loss tensor(4.0908, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.12721354166666668
epoch 2
loss tensor(4.0830, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.13782552083333333
epoch 3
loss tensor(4.0751, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.1466796875
epoch 4
loss tensor(4.0671, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.15348958333333335
epoch 5
loss tensor(4.0591, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.16016927083333332
epoch 6
loss tensor(4.0512, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.16416666666666666
epoch 7
loss tensor(4.0434, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.1679296875
epoch 8
loss tensor(4.0357, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.1715234375
epoch 9
loss tensor(4.0283, device='cuda:0', grad_fn=<DivBackward1>)
accuracy 0.17484375
---------------------------------------